In [13]:
#importing libraries
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [14]:
#Using proxy chrome driver
driver = webdriver.Chrome('C:/Users/DC/Desktop/chromedriver.exe')

In [15]:
#Extracting values,collecting records, dataframe creation
def scrape(address):
    driver.get(address) 
    content=driver.page_source
    soup=BeautifulSoup(content)
    review_links = [r'https://bookpage.com'+i.find('a')['href'] for i in soup.find_all('div', attrs={'class':'read-full'})]

    rows = []

    for review_link in review_links:
        r = driver.get(review_link)
#         time.sleep(5)
        r=driver.page_source
        soup = BeautifulSoup(r,'html.parser')

        rev = soup.find_all('div', attrs={'class':'article-body'})
        review =' '.join([i.text.replace('\n', ' ') for i in rev[0].find_all('p')])

        title_author = []
        details = soup.find_all('div', attrs={'class':'book-data'})

        for t in details:
            try:
                title_author.append(t.find('h4').text.replace('\n', ''))
            except Exception as e:
                pass

        author = title_author[0]
        title = title_author[1]
        genre = [i.text.replace('\n' ,'').replace(' /', ',') for i in soup.find_all('p', attrs={'class':'genre-links'})][0]

        rows.append([title, author, genre, review[1:-1]])

    df = pd.DataFrame(rows, columns=['title', 'author', 'genre', 'reviews'])
    df['title'] = df.title.str.replace('★ ', '')
    df['author'] = df.author.str.replace('★ ', '')
    return df

In [17]:
df=scrape('https://bookpage.com/reviews')

In [18]:
df

,title,author,genre,reviews
0,Crow & Snow,"Robert Broder, Olivier Tallec","Children's, Children's Picture Book",A friendly scarecrow named Crow stands alone ...
1,Magic Dark and Strange,Kelly Powell,"YA Fiction, YA","By day, Catherine works in a print shop. At n..."
2,The Kidnapping Club,Jonathan Daniel Wells,"Nonfiction, American History, African American...","Urbane and bustling, New York City is often c..."
3,Cobble Hill,Cecily von Ziegesar,"Fiction, Popular Fiction","When a person chooses a new home, it’s rarely..."
4,The Zealot and the Emancipator,H.W. Brands,"Nonfiction, History, American History",You might be forgiven for thinking that a boo...
5,A Walk Around the Block,Spike Carlsen,"Nonfiction, Science & Nature",In these unprecedented days of the COVID-19 p...
6,Shadows of the Short Days,Alexander Dan Vilhjálmsson,"Science Fiction & Fantasy, Fantasy","Being in a relationship is tough, even withou..."
7,The Neil Gaiman Reader,Neil Gaiman,"Fiction, Short Stories, Anthology",Neil Gaiman is generally categorized as a wri...
8,Cary Grant,Scott Eyman,"Nonfiction, Biography, Celebrities",Film historian Scott Eyman takes a fresh look...
9,Among the Beasts & Briars,Ashley Poston,"YA, YA Fiction","Cerys lives an idyllic life in the serene, pr..."


In [19]:
#saving to csv
df.to_csv('Book-reviews.csv',index=False,encoding='utf-8')